In [10]:
# !pip install cvzone

  Created wheel for cvzone: filename=cvzone-1.5.6-py3-none-any.whl size=18767 sha256=ae2c1405be0af5caf87284a82c5e183e6d41b35ccf8c2c84f9bdcf04ac53b0ba
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\6f\ec\e5\c576c2f2fa138207498bbbfc8eefb1f0a2efbba7c847742948
Successfully built cvzone


In [1]:
import cv2 as cv
import pickle
import numpy as np
import cvzone

In [ ]:
# img = cv.imread('carParkimg.png')
# cv.imshow('Car parking' , img)
# cv.waitKey(0)

In [2]:
width , height = 110,48
posList = []

try:
    with open('carparkPos' , 'rb' ) as f:
        posList = pickle.load(f)
except:
    posList = []
    
def mouseClick(events, x,  y , flags , params):
    if events == cv.EVENT_LBUTTONDOWN:
        posList.append((x,y))
    if events == cv.EVENT_RBUTTONDOWN:
        for i , pos in enumerate(posList):
            x1 ,y1 = pos 
            if x1 < x < x1 + width and y1 < y < y1 + height:
                posList.pop(i)
    with open('carparkPos' , 'wb') as f:
        pickle.dump(posList , f)

    
# #while True:
# img = cv.imread('carParkimg.png')
# # for pos in posList:
# cv.rectangle(img , (40 , 90 ) , (160 , 145) , (220 , 150 , 198) , 2 )
# cv.imshow("image" , img)
# # cv.setMouseCallback('Image', mouseClick)
# cv.waitKey(0)

while True:
    img = cv.imread('carParkImg.png')
    
    for pos in posList:
#         print("posion is " , pos)
#         print(posList)
        cv.rectangle(img, pos, (pos[0] + width, pos[1] + height), (255, 0, 255), 2)
    
    cv.imshow("Image", img)   
    cv.setMouseCallback("Image", mouseClick)
    cv.waitKey(1)

KeyboardInterrupt: 

## detect the parking space from video 
### load the carpark position from the pickle file and use it in video capture 

In [2]:
cap = cv.VideoCapture('carPark.mp4')

with open('carparkPos' , 'rb' ) as f:
        posList = pickle.load(f)
        
width , height = 110,48

def checkParkingSpace(imgPro):
    
    spaceCounter = 0
    
    for pos in posList:
        x , y = pos
        imgCrop = imgPro[y:y+height , x:x+width]
#         cv.imshow(str(x*y) , imgCrop)
        count = cv.countNonZero(imgCrop)
        cvzone.putTextRect(img , str(count) ,( x , y+height-10),scale = 1 , thickness = 2 , offset = 0)
        
        if count < 900:
            color = (0,255,0)
            thickness = 5
            spaceCounter +=1
        else:
            color = (0,0,255)
            thickness = 5
        cv.rectangle(img, pos, (pos[0] + width, pos[1] + height), color, thickness)
        cvzone.putTextRect(img , f'Free : {str(spaceCounter)}/{len(posList)}' ,(100 , 50),scale = 3 , thickness = 3 , offset = 20 , colorR = (0,200,0))
        
        
while True:
    
    if cap.get(cv.CAP_PROP_POS_FRAMES) == cap.get(cv.CAP_PROP_FRAME_COUNT):
        cap.set(cv.CAP_PROP_POS_FRAMES,0)
    sucess , img = cap.read()
    
    imgGray = cv.cvtColor(img , cv.COLOR_BGR2GRAY)
    imgBlur = cv.GaussianBlur(imgGray , (5,5) , 1)
    imgThreshold = cv.adaptiveThreshold(imgBlur , 255,cv.ADAPTIVE_THRESH_GAUSSIAN_C , cv.THRESH_BINARY_INV , 25 , 16)
    imgMedian = cv.medianBlur(imgThreshold , 5)
    kernel = np.ones((3,3) , np.int8)
    imgDilate = cv.dilate(imgMedian , kernel)
    
    checkParkingSpace(imgDilate)
#     for pos in posList:
#         cv.rectangle(img, pos, (pos[0] + width, pos[1] + height), (255, 0, 255), 2)

    cv.imshow("Video" , img)
    cv.waitKey(1)
    
    

KeyboardInterrupt: 